In [1]:
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np   
import pandas as pd
from scipy import ndimage
import scipy.misc # to visualize only
from scipy.misc import imresize
#from skimage.restoration import denoise_bilateral
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

/home/dongjoon/.local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# Load training data and its label
train_set = np.load('all/train_images.npy', encoding = 'latin1')
test_set = np.load('all/test_images.npy', encoding = 'latin1')
train_label = pd.read_csv('all/train_labels.csv')
train_label = train_label['Category']

# Encode train_label(Category) to integer.
le = preprocessing.LabelEncoder()
le.fit(train_label)
train_label = le.transform(train_label)

# Sanity check.
print(train_set[:,1].shape)
print(train_set)
print(train_label)

# Tried to visualize, kept failing....
'''train_set = train_set.reshape(-1, 100, 100) # reshape 
test_set = test_set.reshape(-1, 100, 100) 
print(train_set.shape)
plt.imshow(train_set[0]) # to visualize only 
train_label.head(20)'''

(10000,)
[[0 array([0., 0., 0., ..., 0., 0., 0.])]
 [1 array([0., 0., 0., ..., 0., 0., 0.])]
 [2 array([0., 0., 0., ..., 0., 0., 0.])]
 ...
 [9997 array([0., 0., 0., ..., 0., 0., 0.])]
 [9998 array([255., 255., 255., ...,   0.,   0.,   0.])]
 [9999 array([0., 0., 0., ..., 0., 0., 0.])]]
[25 20 23 ... 21  3 13]


'train_set = train_set.reshape(-1, 100, 100) # reshape \ntest_set = test_set.reshape(-1, 100, 100) \nprint(train_set.shape)\nplt.imshow(train_set[0]) # to visualize only \ntrain_label.head(20)'

In [3]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 500)

X = pd.DataFrame.from_records(train_set[:,1]) # Convert it into dataframe for model input
X = pca.fit_transform(X) # To reduce dimensionality using PCA as current number of features is way too large.
X = X
y = train_label
y= y
# Split to train and valid set
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.8, random_state=42)
print(x_train.shape)
print(y_train.shape)
x_test = pd.DataFrame.from_records(test_set[:,1])

#x_test.head()

#Instantiate simple machine learning models
log_clf = LogisticRegression(solver='liblinear', random_state=42)
ran_clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
svc_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
    ])
per_clf = Perceptron(random_state=42)


'''per_clf.fit(x_train, y_train)
val_predicted = per_clf.predict(x_val)
print('Validation accuracy: {}'.format(accuracy_score(y_val, val_predicted)))'''


# Train using simple ensemble voting classifier. Doesn't work well.
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', ran_clf), ('svc', svc_clf)],
    voting='hard')

voting_clf.fit(x_train, y_train)

train_predicted = voting_clf.predict(x_train)
val_predicted = voting_clf.predict(x_val)

print('Train accuracy: {}'.format(accuracy_score(y_train, train_predicted)))
#print(f1_score(y_val, val_predicted, average='micro'))
print('Validation accuracy: {}'.format(accuracy_score(y_val, val_predicted)))
print(cross_val_score(voting_clf, x_train, y_train, cv=5, scoring='accuracy'))
# Simple machine learning models for sure doesn't work well
'''ran_clf.fit(x_train, y_train)
predicted = ran_clf.predict(x_val)
print(accuracy_score(y_val, predicted))

svc_clf.fit(x_train, y_train)
predicted = svc_clf.predict(x_val)
print(accuracy_score(y_val, predicted))'''

(2000, 500)
(2000,)


/home/dongjoon/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/dongjoon/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train accuracy: 1.0
Validation accuracy: 0.052875


/home/dongjoon/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/dongjoon/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/dongjoon/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/dongjoon/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

[0.0461165  0.06157635 0.06265664 0.07088608 0.05670103]


/home/dongjoon/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


'ran_clf.fit(x_train, y_train)\npredicted = ran_clf.predict(x_val)\nprint(accuracy_score(y_val, predicted))\n\nsvc_clf.fit(x_train, y_train)\npredicted = svc_clf.predict(x_val)\nprint(accuracy_score(y_val, predicted))'

In [4]:
print(train_predicted)
print(y_train)
print(val_predicted)
print(y_val)
#x_test = pca.fit_transform(x_test)
#test_predicted = voting_clf.predict(x_test)
#print(test_predicted)


[11 20 12 ... 14 27  3]
[11 20 12 ... 14 27  3]
[3 3 3 ... 3 3 3]
[11  4 14 ... 14  9  1]


In [5]:
import tensorflow as tf



ModuleNotFoundError: No module named 'tensorflow'